In [1]:
import pandas as pd
df = pd.read_csv('new_dataset.csv')
df.head()

,valid_time,latitude,longitude,number,expver,u10,v10,d2m,t2m,sp,...,hcc,lcc,mcc,tcc,e,cp,lsp,ptype,sf,z
0,2023-01-01 00:00:00,31.0,77.50,0,1,-0.859558,0.626511,270.64673,273.42407,80197.13,...,0.835052,0.0,0.984222,0.991821,-5.702022e-07,0.0,0.0,0.0,0.000000e+00,19556.635
1,2023-01-01 00:00:00,31.0,77.75,0,1,-0.664246,-0.819778,263.54907,271.98853,79665.13,...,0.889740,0.0,0.968719,0.984161,-3.297115e-06,0.0,0.0,0.0,0.000000e+00,20091.033
2,2023-01-01 00:00:00,31.0,78.00,0,1,-0.497253,-1.058060,254.27562,271.30103,76393.13,...,0.899414,0.0,0.938049,0.961334,-5.457783e-06,0.0,0.0,0.0,0.000000e+00,23481.467
3,2023-01-01 00:00:00,31.0,78.25,0,1,-0.787292,-1.046341,252.62328,267.63500,72034.13,...,0.695007,0.0,0.956085,0.971252,-3.773952e-06,0.0,0.0,0.0,0.000000e+00,28226.463
4,2023-01-01 00:00:00,31.0,78.50,0,1,-0.747253,-1.185989,253.39671,262.30103,66061.13,...,0.311951,0.0,0.987366,0.993134,-5.851034e-07,0.0,0.0,0.0,1.192093e-07,35119.863


In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import datetime
from sklearn.preprocessing import MinMaxScaler

# Load dataset
df = pd.read_csv('dataset/new_dataset.csv')
df['valid_time'] = pd.to_datetime(df['valid_time'])
df.set_index('valid_time', inplace=True)

# Drop unwanted columns
df_clean = df.drop(columns=['latitude', 'longitude', 'number', 'expver'])

# Define columns for which normalization is needed (excluding temperature-related columns)
cols_to_normalize = [
    'u10', 'v10', 'sp',
    'tp', 'skt', 'ssrd', 
    'hcc', 'lcc', 'mcc', 'tcc', 
    'e', 'cp', 'lsp', 
    'ptype', 'sf', 'z'
]

# Normalize the selected columns (leave temperatures in original scale)
scaler = MinMaxScaler()
df_normalized = df_clean.copy()
df_normalized[cols_to_normalize] = scaler.fit_transform(df_clean[cols_to_normalize])

# Prepare data for LSTM (using the scaled dataset)
def create_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(data[i + sequence_length])
    return np.array(sequences), np.array(targets)

sequence_length = 30
scaled_data = df_normalized.values
X, y = create_sequences(scaled_data, sequence_length)

# Split the dataset into training and test sets (80% train, 20% test)
train_size = int(0.8 * len(X))
X_train_full, X_test, y_train_full, y_test = X[:train_size], X[train_size:], y[:train_size], y[train_size:]

# Further split the training data into training and validation sets (10% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(sequence_length, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(X.shape[2]))  # Adjust if you expect a different number of outputs

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

# Save the model after training
model.save('climate_prediction_model.h5')


Epoch 1/10
1035/1035 [==============================] - 44s 39ms/step - loss: 7062.9717 - val_loss: 5346.0137
Epoch 2/10
1035/1035 [==============================] - 46s 45ms/step - loss: 4062.7461 - val_loss: 2929.1011
Epoch 3/10
1035/1035 [==============================] - 44s 43ms/step - loss: 2113.1306 - val_loss: 1392.8944
Epoch 4/10
1035/1035 [==============================] - 45s 44ms/step - loss: 932.2874 - val_loss: 527.0889
Epoch 5/10
1035/1035 [==============================] - 43s 42ms/step - loss: 327.7307 - val_loss: 139.5663
Epoch 6/10
1035/1035 [==============================] - 42s 41ms/step - loss: 99.2374 - val_loss: 26.9570
Epoch 7/10
1035/1035 [==============================] - 42s 41ms/step - loss: 50.0017 - val_loss: 11.4414
Epoch 8/10
1035/1035 [==============================] - 43s 41ms/step - loss: 46.1595 - val_loss: 10.6896
Epoch 9/10
1035/1035 [==============================] - 42s 40ms/step - loss: 45.9610 - val_loss: 10.6484
Epoch 10/10
1035/1035 [=======

C:\Users\Charu Gupta\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Final evaluation on the test set
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")


575/575 [==============================] - 5s 9ms/step - loss: 11.5391
Test Loss: 11.53906536102295
